## Running High Court scraper

In [1]:
!pip install -U soscipy

  Using cached soscipy-0.0.29-py3-none-any.whl (21 kB)
  Using cached seaborn-0.11.1-py3-none-any.whl (285 kB)
  Using cached urllib3-1.26.4-py2.py3-none-any.whl (153 kB)
  Using cached pip-21.0.1-py3-none-any.whl (1.5 MB)
  Using cached pandas-1.2.4-cp38-cp38-macosx_10_9_x86_64.whl (10.5 MB)
  Using cached attrs-20.3.0-py2.py3-none-any.whl (49 kB)
  Using cached pytest-6.2.3-py3-none-any.whl (280 kB)
  Using cached setuptools-52.0.0-py3-none-any.whl (784 kB)
  Using cached wheel-0.36.2-py2.py3-none-any.whl (35 kB)
  Using cached tornado-6.1-cp38-cp38-macosx_10_9_x86_64.whl (416 kB)
  Using cached py-1.10.0-py2.py3-none-any.whl (97 kB)
ERROR: Could not find a version that satisfies the requirement openssl~=1.1.1 (from soscipy) (from versions: none)
ERROR: No matching distribution found for openssl~=1.1.1 (from soscipy)


In [2]:
from soscipy.fetch import HighCourt

In [4]:
x = HighCourt.HC_list

In [8]:
'Delhi' in x

True